# What Gifts will Santa be giving to the Kids?

Looking at the problem, it seems to be a combinatorics problem similar to the knapsack problem. Thus I will use Google's OR-Tools to solve this system. Afterwards, I will use the Hungarian algorithm (also called Hungarian Method) to improve the solution. I could simply use the Hungarian Method, however it is O(n<sup>3</sup>). This would take absurdly long so we get closer to an optimal solution with the OR-tools then improve with the hunagian algorithm

In [ ]:
import numpy as np
import pandas as pd
from ortools.graph import pywrapgraph

INPUT_PATH = '../input/'

In [ ]:
def avg_normalized_happiness(pred, gift, wish):
    n_children = 1000000  # n children to give
    n_gift_type = 1000  # n types of gifts available
    twins = int(0.004 * n_children)  # 0.4% of all population, rounded to the closest even number

    # check if twins have the same gift
    for t1 in range(0, twins, 2):
        twin1 = pred[t1]
        twin2 = pred[t1 + 1]
        assert twin1 == twin2

    total_child_happiness = 0
    total_gift_happiness = 0

    for i in range(len(pred)):
        child_id = i
        gift_id = pred[i]

        # check if child_id and gift_id exist
        assert 0 <= child_id < n_children
        assert 0 <= gift_id < n_gift_type
        child_happiness = 20 - 2 * np.where(wish[child_id] == gift_id)[0]
        if not child_happiness:
            child_happiness = -1

        gift_happiness = 2000 - 2 * np.where(gift[gift_id] == child_id)[0]
        if not gift_happiness:
            gift_happiness = -1

        total_child_happiness += child_happiness
        total_gift_happiness += gift_happiness

    # print(max_child_happiness, max_gift_happiness
    child_h = float(total_child_happiness) / 20000000
    santa_h = float(total_gift_happiness) / 2000000000
    print('Normalized child happiness: ', child_h)
    print('Normalized santa happiness: ', santa_h)
    return child_h + santa_h

In [ ]:
def get_overall_hapiness(wish, gift):

    res_child = dict()
    for i in range(0, 4000, 2):
        for j in range(wish.shape[1]):
            res_child[(i, wish[i][j])] = 50*(1 + (wish.shape[1] - j) * 2)
            res_child[(i + 1, wish[i][j])] = 50*(1 + (wish.shape[1] - j) * 2)
        for j in range(wish.shape[1]):
            if (i, wish[i+1][j]) in res_child:
                res_child[(i, wish[i+1][j])] += 50*(1 + (wish.shape[1] - j) * 2)
            else:
                res_child[(i, wish[i + 1][j])] = 50*(1 + (wish.shape[1] - j) * 2)
            if (i+1, wish[i + 1][j]) in res_child:
                res_child[(i + 1, wish[i+1][j])] += 50*(1 + (wish.shape[1] - j) * 2)
            else:
                res_child[(i + 1, wish[i + 1][j])] = 50*(1 + (wish.shape[1] - j) * 2)

    for i in range(4000, wish.shape[0]):
        for j in range(wish.shape[1]):
            res_child[(i, wish[i][j])] = 100*(1 + (wish.shape[1] - j)*2)

    res_santa = dict()
    for i in range(gift.shape[0]):
        for j in range(gift.shape[1]):
            res_santa[(gift[i][j], i)] = (1 + (gift.shape[1] - j)*2)

    positive_cases = list(set(res_santa.keys()) | set(res_child.keys()))
    print('Positive case tuples (child, gift): {}'.format(len(positive_cases)))

    res = dict()
    for p in positive_cases:
        res[p] = 0
        if p in res_child:
            res[p] += res_child[p]
        if p in res_santa:
            res[p] += res_santa[p]
    return res

In [ ]:
def solve():
    wish = pd.read_csv(INPUT_PATH + 'child_wishlist.csv', header=None).as_matrix()[:, 1:]
    gift_init = pd.read_csv(INPUT_PATH + 'gift_goodkids.csv', header=None).as_matrix()[:, 1:]
    gift = gift_init.copy()
    answ = np.zeros(len(wish), dtype=np.int32)
    answ[:] = -1
    gift_count = np.zeros(len(gift), dtype=np.int32)
    happiness = get_overall_hapiness(wish, gift)

    start_nodes = []
    end_nodes = []
    capacities = []
    unit_costs = []
    supplies = []

    for h in happiness:
        c, g = h
        # print(c, g, happiness[h])
        start_nodes.append(int(c))
        end_nodes.append(int(1000000 + g))
        capacities.append(1)
        unit_costs.append(-happiness[h])

    for i in range(1000000):
        supplies.append(1)
    for j in range(1000000, 1001000):
        supplies.append(-1000)

    # Instantiate a SimpleMinCostFlow solver.
    min_cost_flow = pywrapgraph.SimpleMinCostFlow()

    # Add each arc.
    for i in range(0, len(start_nodes)):
        min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i], capacities[i], unit_costs[i])

    # Add node supplies.
    for i in range(0, len(supplies)):
        min_cost_flow.SetNodeSupply(i, supplies[i])

    # Find the minimum cost flow
    print('Start solve....')
    min_cost_flow.SolveMaxFlowWithMinCost()
    res1 = min_cost_flow.MaximumFlow()
    print('Maximum flow:', res1)
    res2 = min_cost_flow.OptimalCost()
    print('Optimal cost:', -res2 / 2000000000)
    print('Num arcs:', min_cost_flow.NumArcs())

    total = 0
    for i in range(min_cost_flow.NumArcs()):
        cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
        if cost != 0:
            answ[min_cost_flow.Tail(i)] = min_cost_flow.Head(i) - 1000000
    print('Assigned: {}'.format(total))

    # instead of improving twins only by checking the first one, check both and 
    # compare the happiness score
    for i in range(0, 4000, 2):
        print('Improve twin {}'.format(i))
        if answ[i] != answ[i+1]:
            less_hapiness = 1000000000
            worst_child = [-1,-1]
            for k in [i, i+1]:
                for j in range(4000, 1000000):
                    if answ[j] == answ[k]:
                        if (j, answ[k]) in happiness:
                            score = happiness[(j, answ[k])]
                            if score < less_hapiness:
                                less_hapiness = score
                                if k == i:
                                    worst_child[0] = j
                                else:
                                    worst_child[1] = j
                        else:
                            if k == i:
                                worst_child[0] = j
                            else:
                                worst_child[1] = j
                            break
            if   0 < worst_child[0] and (worst_child[1] == -1 or worst_child[0] < worst_child[1]):
                    answ[worst_child[0]] = answ[i+1]
                    answ[i+1] = answ[i]
            elif 0 < worst_child[1] and (worst_child[0] == -1 or worst_child[1] < worst_child[0]):
                    answ[worst_child[1]] = answ[i]
                    answ[i] = answ[i+1]
    
    if answ.min() == -1:
        print('Some children without present')
        exit()

    if gift_count.max() > 1000:
        print('Some error in kernel: {}'.format(gift_count.max()))
        exit()

    print('Start score calculation...')
    score = avg_normalized_happiness(answ, gift_init, wish)
    print('Predicted score: {:.8f}'.format(score))

    out = open('FinalSub.csv', 'w')
    out.write('ChildId,GiftId\n')
    for i in range(len(answ)):
        out.write(str(i) + ',' + str(answ[i]) + '\n')
    out.close()

In [ ]:
solve()

# Improving the Solution with Hungarian Algorithm

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment
import datetime as dt
from collections import defaultdict, Counter
from tqdm import tqdm
import matplotlib.pyplot as plt
import datetime as dt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 16
import seaborn as sns

In [ ]:
N_CHILDREN = 1000000
N_GIFT_TYPE = 1000
N_GIFT_QUANTITY = 1000
N_GIFT_PREF = 1000
N_CHILD_PREF = 10
TWINS = int(0.004 * N_CHILDREN)

CHILD_PREF = pd.read_csv('../input/santa-gift-matching/child_wishlist.csv', header=None).drop(0, 1).values
GIFT_PREF = pd.read_csv('../input/santa-gift-matching/gift_goodkids.csv', header=None).drop(0, 1).values

In [ ]:
GIFT_HAPPINESS = {}
for g in range(N_GIFT_TYPE):
    GIFT_HAPPINESS[g] = defaultdict(lambda: 1. / (2 * N_GIFT_PREF))
    for i, c in enumerate(GIFT_PREF[g]):
        GIFT_HAPPINESS[g][c] = -1. * (N_GIFT_PREF - i) / N_GIFT_PREF

CHILD_HAPPINESS = {}
for c in range(N_CHILDREN):
    CHILD_HAPPINESS[c] = defaultdict(lambda: 1. / (2 * N_CHILD_PREF))
    for i, g in enumerate(CHILD_PREF[c]):
        CHILD_HAPPINESS[c][g] = -1. * (N_CHILD_PREF - i) / N_CHILD_PREF

GIFT_IDS = np.array([[g] * N_GIFT_QUANTITY for g in range(N_GIFT_TYPE)]).flatten()

In [ ]:
def my_avg_normalized_happiness(pred):
    total_child_happiness = 0
    total_gift_happiness = np.zeros(1000)
    for c, g in pred:
        total_child_happiness +=  -CHILD_HAPPINESS[c][g]
        total_gift_happiness[g] += -GIFT_HAPPINESS[g][c]
    nch = total_child_happiness / N_CHILDREN
    ngh = np.mean(total_gift_happiness) / 1000
    print('normalized child happiness', nch)
    print('normalized gift happiness', ngh)
    return nch + ngh

In [ ]:
def optimize_block(child_block, current_gift_ids):
    gift_block = current_gift_ids[child_block]
    C = np.zeros((BLOCK_SIZE, BLOCK_SIZE))
    for i in range(BLOCK_SIZE):
        c = child_block[i]
        for j in range(BLOCK_SIZE):
            g = GIFT_IDS[gift_block[j]]
            C[i, j] = CHILD_HAPPINESS[c][g] + GIFT_HAPPINESS[g][c]
    row_ind, col_ind = linear_sum_assignment(C)
    return (child_block[row_ind], gift_block[col_ind])

In [ ]:
BLOCK_SIZE = 400
INITIAL_SUBMISSION = './FinalSub.csv'
N_BLOCKS = (N_CHILDREN - TWINS) / BLOCK_SIZE
print('Block size: {}, n_blocks {}'.format(BLOCK_SIZE, N_BLOCKS))

In [ ]:
subm = pd.read_csv(INITIAL_SUBMISSION)
initial_anh = my_avg_normalized_happiness(subm[['ChildId', 'GiftId']].values.tolist())
print(initial_anh)
subm['gift_rank'] = subm.groupby('GiftId').rank() - 1
subm['gift_id'] = subm['GiftId'] * 1000 + subm['gift_rank']
subm['gift_id'] = subm['gift_id'].astype(np.int64)
current_gift_ids = subm['gift_id'].values

In [ ]:
start_time = dt.datetime.now()
for i in range(1):
    child_blocks = np.split(np.random.permutation(range(TWINS, N_CHILDREN)), N_BLOCKS)
    for child_block in tqdm(child_blocks[:500]):
        cids, gids = optimize_block(child_block, current_gift_ids=current_gift_ids)
        current_gift_ids[cids] = gids
    subm['GiftId'] = GIFT_IDS[current_gift_ids]
    anh = my_avg_normalized_happiness(subm[['ChildId', 'GiftId']].values.tolist())
    end_time = dt.datetime.now()
    print(i, anh, (end_time-start_time).total_seconds())
subm[['ChildId', 'GiftId']].to_csv('./submission_%i.csv' % int(anh * 10 ** 6), index=False)

In [ ]:
print('Improvement {}'.format(anh - initial_anh))

In [ ]:
'{:.1f} hours required to reach the top.'.format(((0.94253901 - 0.93421513) / (0.93421513 - initial_anh)) * 8)

# Happiness Distribution

In [ ]:
child_happiness = np.zeros(N_CHILDREN)
gift_happiness = np.zeros(N_CHILDREN)
for (c, g) in subm[['ChildId', 'GiftId']].values.tolist():
    child_happiness[c] += -CHILD_HAPPINESS[c][g]
    gift_happiness[c] += -GIFT_HAPPINESS[g][c]

In [ ]:
plt.hist(gift_happiness, bins=20, color='r', normed=True, alpha=0.5, label='Santa happiness')
plt.hist(child_happiness, bins=20, color='g', normed=True, alpha=0.5, label='Child happiness')
plt.legend(loc=0)
plt.grid()
plt.xlabel('Happiness')
plt.title('The children will be happier than Santa!')
plt.show();

# Time Complexity

In [ ]:
result = []
for n in np.arange(100, 1600, 100):
    C = np.random.random((n, n))
    st = dt.datetime.now()
    linear_sum_assignment(C)
    et = dt.datetime.now()
    result.append([n, (et - st).total_seconds()])

In [ ]:
result = np.array(result)
poly_estimate = np.polyfit(result[:, 0], result[:, 1], 3)

In [ ]:
plt.scatter(result[:, 0], result[:, 1], c='y', s=500, marker='*', label='Run time')
plt.plot(result[:, 0], np.poly1d(poly_estimate)(result[:, 0]), c='g', lw=3, label='Polynomial Estimate')
plt.xlabel('Number of vertices')
plt.ylabel('Run time (s)')
plt.grid()
plt.title('Hungarian method - O(n^3) time complexity')
plt.legend(loc=0)
plt.show()